# 前処理

## データ読み込み

In [ ]:
import cv2
import numpy as np
from glob import glob

num_classes = 2
img_height, img_width = 64, 64

CLS = ['akahara', 'madara']

def data_load(path, hf=False, vf=False, rot=False):
    xs = []
    ts = []
    paths = []

    # answer your code
    for dir_path in glob(path + '/*'):
        for path in glob(dir_path + '/*'):
            x = cv2.imread(path) # (height, width, c(BGR))
            x = cv2.resize(x, (img_width, img_height)).astype(np.float32) # to 64×64
            x /= 255.
            xs.append(xs)
            
            t =  0 if CLS[0] in path else 1  
            ts.append(t)
            paths.append(t)
            
            # 左右反転
            if hf:
                xs.append(x[:,::-1]) 
                ts.append(t)
                paths.append(path)
                
            # 上下反転
            if vf:
                xs.append(x[::-1]) 
                ts.append(t)
                paths.append(path)
            
            # 上下・左右反転
            if vf and hf:
                xs.append(x[::-1,::-1]) 
                ts.append(t)
                paths.append(path)
                
    xs = np.array(xs, dtype=np.float32)        
    ts = np.array(ts, dtype=np.int)
    
    # pytrochのinput [データ数、チャネル、画像の縦サイズ、画像の横サイズ]
    xs = xs.transpose(0,3,1,2)
    
    return xs, ts, paths

xs, ts, paths = data_load('./Dataset/train/images/')

In [ ]:
xs.shape

## ミニバッチ

In [22]:
mb = 3
mbi = 0
train_ind = np.arange(len(xs))
np.random.seed(0)
np.random.shuffle(train_ind)

epoch_max = 3
epoch = 0

while epoch < epoch_max:
    if mbi + mb > len(xs):
        mb_ind = train_ind[mbi:]
        np.random.shuffle(train_ind)
        # hstackはaxis=1が違うとき, vstackはaxis=0が違うとき可能
        mb_ind = np.hstack((mb_ind, train_ind[:(mb-(len(xs)-mbi))]))
        mbi = mb - (len(xs)-mbi)
        epoch += 1
    else:
        mb_ind = train_ind[mbi: mbi+mb]
        mbi += mb
            
    print(mb_ind)

[1 6 8]
[ 9 13  4]
[ 2 14 10]
[ 7 15 11]
[3 0 5]
[6 8 5]
[3 7 1]
[ 9 13 15]
[11  4 12]
[10  0 14]
[8 6 9]
[14 15  1]
[12  2 13]
[10  4  0]
[ 5 11  7]
[3 8 6]
[11  4 14]


In [17]:
xs[0].shape

(3, 64, 64)